In [1]:
import os, sys
import time
import copy
import random
import numpy as np
from sklearn.utils import shuffle

In [2]:
with open("../llvmpasses.txt", mode="r") as f:
    PASSES = [x.strip() for x in f.readlines()]

In [3]:
PASSES[:5]

['-openmp-opt-cgscc',
 '-early-cse-memssa',
 '-strip-dead-prototypes',
 '-simplifycfg',
 '-memcpyopt']

In [4]:
pg_path = "../polybench/linear-algebra/blas/symm/symm.c"
pg_name = "symm"

In [5]:
ini_cmd1 = 'clang -O0 -emit-llvm -I ../polybench/utilities -I ../polybench/datamining/correlation/correlation -c {path} -o {name}.bc'
ini_cmd2 = 'clang -O0 -emit-llvm -I ../polybench/utilities -I ../polybench/datamining/correlation/correlation -c ../polybench/utilities/polybench.c -o polybench.bc;'
cmd1 = ' -S polybench.bc -o polybench.opt.bc; llc -O0 -filetype=obj polybench.opt.bc -o polybench.o'
cmd2 = ' -S {name}.bc -o {name}.opt.bc; llc -O0 -filetype=obj {name}.opt.bc -o {name}.o'
cmd3 = 'clang -O0 -no-pie -lm *.o -o {name}'
cmd4 = './{name}'
clean_cmd = 'rm -rf *bc *.o *.I *.S {name}'

In [6]:
for iter in range(10):
    rand_passes = shuffle(PASSES)
    print(f"Iteration {iter + 1}:", " ".join(rand_passes))
    print("*" * 5)
    os.system(clean_cmd.format(name=pg_name))
    os.system(ini_cmd1.format(path=pg_path, name=pg_name))
    os.system(ini_cmd2)
    os.system("opt -O0 " + " ".join(rand_passes) + cmd1)
    os.system("opt -O0 " + " ".join(rand_passes) + cmd2.format(name=pg_name))
    print(cmd3.format(name=pg_name))
    if os.system(cmd3.format(name=pg_name)) == 0:
        print("Compiled! Measuring execution time...")
    exe_times = []
    for rtime in range(10):
        begin = time.time()
        ret = os.system(cmd4.format(name=pg_name))
        if ret > 0:
            continue
        end = time.time()
        print(f"R{rtime + 1}:", end - begin)
        os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
        exe_times.append(end - begin)
    print(exe_times)
    print(f"Std: {np.std(exe_times)}; Median: {np.median(exe_times)}; Mean: {np.mean(exe_times)}; Min: {np.min(exe_times)}; Max: {np.max(exe_times)}")
    print("=" * 20)

Iteration 1: -constmerge -forceattrs -block-freq -loop-deletion -cg-profile -slp-vectorizer -callsite-splitting -lcssa -inline -lower-constant-intrinsics -scoped-noalias-aa -postdomtree -function-attrs -annotation2metadata -targetlibinfo -ipsccp -mldst-motion -loop-distribute -inferattrs -memoryssa -dse -pgo-memop-opt -elim-avail-extern -aggressive-instcombine -alignment-from-assumptions -mem2reg -memdep -memcpyopt -annotation-remarks -lower-expect -simplifycfg -domtree -branch-prob -demanded-bits -transform-warning -lazy-value-info -gvn -loop-load-elim -openmp-opt-cgscc -loop-sink -loops -early-cse-memssa -libcalls-shrinkwrap -globaldce -verify -jump-threading -float2int -loop-rotate -reassociate -loop-simplify -tailcallelim -loop-vectorize -phi-values -adce -deadargelim -inject-tli-mappings -called-value-propagation -globalopt -early-cse -licm -instsimplify -basic-aa -indvars -sroa -bdce -loop-idiom -instcombine -loop-unroll -correlated-propagation -div-rem-pairs -speculative-executi

R9: 25.664873600006104
